## Script used to download my personal stats from my favorite mobile game Clash of Clans  
Author: Luis Mejia
<ol>
  <li>Imports and declarations</li>
  <li>Make request to Clash of Clans API</li>
  <li>Scrape JSON file and load data into their own DataFrame</li>
  <li>Loop through the DataFrames and insert them into my personal Google BigQuery project</li>
</ol>

In [ ]:
import requests
import json
import pandas as pd
from datetime import date
from tabulate import tabulate
from google.cloud import bigquery
import warnings

warnings.filterwarnings("ignore")
api_token = 'my clash api token'
players = ['L29CPQL8', '2RV8YG8U', '8PP0PJ8Q','LG0VGQYC','2P8CYY9VV', '2Y2G8YUJV', 'JJ98YLJG', '2QJQYY9VR','G29J22YG','9P9PCL9V','2QPC8PPG','2Q28PJYG','28P9VYYLP','29PY0JQL','8YJ0JJPL','JQJCJQLG', 'CC8G0CY8']
today = date.today()
# client = bigquery.Client.from_service_account_json('service_account.json')

In [ ]:
#Make COC API request for each player in the players list
try:
    achievements_list = []
    troops_list = []
    heroes_list = []
    spells_list = []      
    play_stats_list = []                   
    for player in players:       
        url = "https://api.clashofclans.com/v1/players/%23{player}".format(player=player)
        r = requests.get(url, headers={"Accept":"application/json", "authorization":"Bearer "+api_token}) #, params = {"limit":20})
        data = json.loads(r.text)

        #Get achievements data
        achievements = pd.DataFrame(data['achievements'])
        achievements[['archive_date','player_tag']] = pd.DataFrame([[today,player]],index=achievements.index)
        achievements_list.append(achievements)

        #Get troops stats
        troops = pd.DataFrame(data['troops'])
        troops[['archive_date','player_tag']] = pd.DataFrame([[today,player]],index=troops.index)
        troops_list.append(troops)

        #Get heroes data
        heroes = pd.DataFrame(data['heroes'])
        heroes[['archive_date','player_tag']] = pd.DataFrame([[today,player]],index=heroes.index)
        heroes_list.append(heroes)

        #Get Spells data
        spells = pd.DataFrame(data['spells'])
        spells[['archive_date','player_tag']] = pd.DataFrame([[today,player]],index=spells.index)
        spells_list.append(spells)

        #Get general player stats
        play_stats = pd.DataFrame.from_dict(data, orient='index')
        play_stats = play_stats.transpose()
        # play_stats = play_stats[play_stats.columns[:19]]
        play_stats[['archive_date','player_tag']] = pd.DataFrame([[today,player]],index=play_stats.index)   
        play_stats_list.append(play_stats)         


    df_achievements = pd.concat(achievements_list)
    df_troops = pd.concat(troops_list)
    df_heroes = pd.concat(heroes_list)
    df_spells = pd.concat(spells_list)
    df_play_stats = pd.concat(play_stats_list)

    #Give dataframes names, based on names, a different table will be updated in BigQuery
    df_achievements.Name = 'achievements'
    df_troops.Name = 'troops'
    df_heroes.Name = 'heroes'
    df_spells.Name = 'spells'
    df_play_stats.Name = 'play_stats'
except Exception as e:
    print(e)    

In [14]:
def import_to_gbq(df, destination_table):
    try:
        ## If you want the column and row count then
        table_id = destination_table
        job_config = bigquery.LoadJobConfig(
            schema=[
            ],
            write_disposition="WRITE_TRUNCATE",
        )

        job = client.load_table_from_dataframe(
            df, table_id, job_config=job_config
        )  # Make an API request.

        job.result()  # Wait for the job to complete.
        table = client.get_table(table_id)  # Make an API request.
        print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))
    except Exception as e:
        print(e)

In [17]:
#loop through all frames and insert them in GBQ
all_frames = [df_achievements,df_troops,df_heroes,df_spells,df_play_stats]
for frame in all_frames:
    frame_name = frame.Name
    if frame_name == 'play_stats':
        # print(tabulate(frame.head(10), headers='keys', tablefmt='simple_outline')) ##fancy_outline, simple_outline, simple_grid, rounded_grid, fancy_grid, html
        import_to_gbq(frame, 'fleet-parser-330316.luis_stage.stg_coc_player_stats')
    elif frame_name == 'achievements':
        # print(tabulate(frame.head(10), headers='keys', tablefmt='simple_outline')) ##fancy_outline, simple_outline, simple_grid, rounded_grid, fancy_grid, html
        import_to_gbq(frame, 'fleet-parser-330316.luis_stage.stg_coc_player_achievements')
    elif frame_name == 'troops':
        print(tabulate(frame.head(10), headers='keys', tablefmt='simple_outline')) ##fancy_outline, simple_outline, simple_grid, rounded_grid, fancy_grid, html
        import_to_gbq(frame, 'fleet-parser-330316.luis_stage.stg_coc_player_troops')
    elif frame_name == 'heroes':
        print(tabulate(frame.head(10), headers='keys', tablefmt='simple_outline')) ##fancy_outline, simple_outline, simple_grid, rounded_grid, fancy_grid, html
        import_to_gbq(frame, 'fleet-parser-330316.luis_stage.stg_coc_player_heroes')
    elif frame_name == 'spells':
        print(tabulate(frame.head(10), headers='keys', tablefmt='simple_outline')) ##fancy_outline, simple_outline, simple_grid, rounded_grid, fancy_grid, html
        import_to_gbq(frame, 'fleet-parser-330316.luis_stage.stg_coc_player_spells')
    else:
        'nothing'                        
    

Loaded 782 rows and 9 columns to fleet-parser-330316.luis_stage.stg_coc_player_achievements
┌────┬──────────────┬─────────┬────────────┬───────────┬────────────────┬──────────────┬──────────────────────┐
│    │ name         │   level │   maxLevel │ village   │ archive_date   │ player_tag   │   superTroopIsActive │
├────┼──────────────┼─────────┼────────────┼───────────┼────────────────┼──────────────┼──────────────────────┤
│  0 │ Barbarian    │      11 │         11 │ home      │ 2023-02-02     │ L29CPQL8     │                  nan │
│  1 │ Archer       │      11 │         11 │ home      │ 2023-02-02     │ L29CPQL8     │                  nan │
│  2 │ Goblin       │       8 │          8 │ home      │ 2023-02-02     │ L29CPQL8     │                  nan │
│  3 │ Giant        │      11 │         11 │ home      │ 2023-02-02     │ L29CPQL8     │                  nan │
│  4 │ Wall Breaker │      11 │         11 │ home      │ 2023-02-02     │ L29CPQL8     │                  nan │
│  5 │ Ballo

In [18]:
#Finally, run a Stored Procedure that will insert all raw data from staging tables into prod tables
try:
    query_job = client.query('CALL `fleet-parser-330316.luistest.sp_coc_player_data_import`();')
    results = query_job.result()

    if query_job.state == 'DONE':
        print('updated all tables')
except Exception as e:
    print(e)

updated all tables


# Thank you!